<a href="https://colab.research.google.com/github/AmanPriyanshu/Weird-Keras-Models/blob/master/weird_keras_models1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTS:

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Input, BatchNormalization, Add
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D, Concatenate, Dropout

## BASE MODEL:

In [23]:
def convBlock(x):
  conv = Conv2D(128, (3,3), strides=1, padding="same", activation='relu')(x)
  bn = BatchNormalization()(conv)
  return bn

def ResBlock(x):
  conv1 = Conv2D(128, kernel_size=(3,3), strides=1, padding="same", activation='relu')(x)
  bn1 = BatchNormalization()(conv1)
  conv2 = Conv2D(128, kernel_size=(3,3), strides=1, padding="same", activation='relu')(bn1)
  bn2 = BatchNormalization()(conv2)
  out = Add()([bn2, x]) 
  return out

def OutBlock(x):
  conv = Conv2D(3, kernel_size=(1,1), strides=1, activation='relu')(x)
  bn = BatchNormalization()(conv)
  flat = Flatten()(bn)
  dense1 = Dense((32), activation='relu')(flat)
  value = Dense(1, activation='tanh')(dense1) #Value Head
  
  conv = Conv2D(32, kernel_size=(1,1), strides=1, activation='relu')(x)
  bn = BatchNormalization()(conv)
  flat = Flatten()(bn)
  linear = Dense(6*7*32, activation='relu')(flat)
  policy = Dense(7, activation='softmax')(linear)

  return policy, value

In [24]:
input0 = Input([6, 7, 3])
model = convBlock(input0)
for _ in range(19):
  model = ResBlock(model)
policy, value = OutBlock(model)

model = Model(inputs=input0, outputs=[policy, value])

In [25]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 6, 7, 3)]    0                                            
__________________________________________________________________________________________________
conv2d_165 (Conv2D)             (None, 6, 7, 128)    3584        input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_163 (BatchN (None, 6, 7, 128)    512         conv2d_165[0][0]                 
__________________________________________________________________________________________________
conv2d_166 (Conv2D)             (None, 6, 7, 128)    147584      batch_normalization_163[0][0]    
_______________________________________________________________________________________

## Custom Loss Function

In [41]:
losses = {
	"category_output": "categorical_crossentropy",
	"color_output": "categorical_crossentropy",
}
lossWeights = {"category_output": 1.0, "color_output": 1.0}

In [49]:
model.compile(optimizer='adam', loss=losses, loss_weights=lossWeights)